# NYC Electricity time series prediction
# Research Question

# Algorithm

Use simplex forecasting from Emperical Data Modelling to predict power levels.


In [1]:
#!pip install kedm plotly jupyter-dash

In [2]:
import cudf

In [3]:
df_forecast = cudf.read_csv('http://mis.nyiso.com/public/csv/isolf/20210725isolf.csv'); df_forecast
df_forecast.to_csv('forecast.csv')

In [4]:
df_actual = cudf.read_csv('http://mis.nyiso.com/public/csv/pal/20210720pal.csv')
for i in range(21,26):
    df_actual = df_actual.append(cudf.read_csv('http://mis.nyiso.com/public/csv/pal/202107'+ str(i) + 'pal.csv'))
df_actual.to_csv('df_actual.csv')
df_result = cudf.read_csv('http://mis.nyiso.com/public/csv/pal/20210726pal.csv')
df_result.to_csv('df_result.csv')

In [5]:
from datetime import datetime
df_nyc = df_actual[df_actual['Name']=='N.Y.C.']
df_nyc_result = df_result[df_result['Name']=='N.Y.C.']
df_nyc['Time Stamp'] =cudf.to_datetime(df_nyc['Time Stamp'])
df_nyc_result['Time Stamp'] =cudf.to_datetime(df_nyc_result['Time Stamp'])
nyc_timeseries = df_nyc[['Time Stamp', 'Load']]
nyc_ts_result = df_nyc_result[['Time Stamp', 'Load']]

So now we got a data series for NYC. Let's run it through kEDM! 

Keichi Takahashi, Wassapon Watanakeesuntorn, Kohei Ichikawa, Joseph Park, Ryousei Takano, Jason Haga, George Sugihara, Gerald M. Pao, "kEDM: A Performance-portable Implementation of Empirical Dynamical Modeling," Practice & Experience in Advanced Research Computing (PEARC 2021), Jul. 2021.

First, what's the optimal embedding dimension?

In [6]:
import kedm
import numpy as np
ts = (nyc_timeseries['Load']).to_array()
ts_result = (nyc_ts_result['Load']).to_array()
%time
tau_num = 1
pred_interval = len(ts)//4
dim = kedm.edim(ts, tau=tau_num, E_max=50, Tp=pred_interval ); dim

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 7.15 µs


1

In [7]:
%time
e = dim
ts_lib = ts[0:len(ts)//2]
ts_target = ts[len(ts)//2:-1]
tdf = cudf.DataFrame()
ordered_series = []
yoffset = 100
def update_df(e=e, ts_lib=ts_lib, ts_target=ts_target, pred_interval=pred_interval, tau_num=tau_num):
    global ordered_series, tdf, ts_labels, yoffset
    import time
    import kedm
    ts_simplex = kedm.simplex(library=ts_lib, target=ts_target, E = e, Tp = pred_interval, tau=tau_num) + yoffset
    ts_simplex_r = kedm.simplex_eval(library=ts_lib, target=ts_target, E = e, Tp = pred_interval, tau=tau_num)

    ts_smap = kedm.smap(library=ts_lib,target=ts_target,E = e, Tp=pred_interval, tau=tau_num ) + yoffset
    
    ts_smap_r = kedm.smap_eval(library=ts_lib,target=ts_target,E = e, Tp=pred_interval, tau=tau_num )

    nyc_tstamps = nyc_timeseries['Time Stamp']
    nyc_tstamps_res = nyc_ts_result['Time Stamp']
    print(ts_simplex_r, ts_smap_r)
    
    timedelta = nyc_tstamps.iloc[2]- nyc_tstamps.iloc[1]
    #"static" timewarping by adding an offset at item index as opposed to dynamic timewarping
    max_len_pred = max(len(ts_simplex), len(ts_smap))
    pred_tstamps = [nyc_tstamps.iloc[-1] + (timedelta * n) - (timedelta * ((max_len_pred-pred_interval))) for n in range(0,max_len_pred)]
    ts_forecast = df_forecast['N.Y.C.'].to_array()
    forecast_tstamps = df_forecast['Time Stamp'].to_array()
    
    tseries=[ts, ts_result, ts_simplex, ts_smap, ts_forecast ]
    tstamps=[nyc_tstamps, nyc_tstamps_res, pred_tstamps, pred_tstamps, forecast_tstamps]
    ts_labels=['Timeseries', 'Result', 'Simplex r:' + str(ts_simplex_r), 'Smap r:' + str(ts_smap_r), 'Forecast']
    ordered_series = list(zip(tseries, tstamps))
    max_len = max([ max(len(s[0]),len(s[1])) for s in ordered_series])
    
    for i, (s, stamp_r) in enumerate(ordered_series):
        stamp = cudf.Series(cudf.to_datetime(stamp_r))
        tdf['y' + str(i)] = np.append(s, [0] * (max_len-len(s)))
        addl_ts = [nyc_tstamps.iloc[-1] + (timedelta * i) for i in range(0, (max_len-len(s)))]
        stamp_np = [s for s in stamp.values_host]
        tdf['x' + str(i)] = np.append(np.array(stamp_np, dtype='datetime64'),np.array(addl_ts, dtype='datetime64'))

update_df()

CPU times: user 3 µs, sys: 2 µs, total: 5 µs
Wall time: 7.87 µs
0.7910515069961548 0.823871374130249


In [8]:
#!pip install jupyter-dash tables plotly
#!pip install dash dash-bootstrap-components dash-html-components matplotlib plotly

In [9]:

import json

import cudf
import cupy as cp
import cupy
import plotly.graph_objects as go

import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State

from flask import request
from jupyter_dash import JupyterDash
from IPython.display import display, HTML

In [10]:
def plot_main_figure(gdf):
    fig = go.Figure()
    for i in range(0, len(list(ordered_series))):
        si = str(i)
        fig=fig.add_trace(
            go.Scattergl({
            'x': gdf['x' + str(i)].to_array(),
            'y': gdf['y' + str(i)].to_array(),
            'name': ts_labels[i],
            'mode': 'markers',
            'marker': {'size': 4}
        }))
    # fig.update_layout(showlegend=True, clickmode='event+select')
    fig.update_layout(showlegend=True, clickmode='event', dragmode='select')
    return fig

In [11]:
# Stylesheet for the application
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
# Please change the proxy_port if the port is already in use.
proxy_port = 8050

# Define the application.
# use this for when you are on more secured instances
app = JupyterDash(__name__,
                  external_stylesheets=external_stylesheets,
                  requests_pathname_prefix='/proxy/' + str(proxy_port) + '/')

# use this is for when you control all the ports
#app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# Styles tobe used
styles = {
    'pre': {
        'border': 'thin lightgrey solid',
        'overflowX': 'scroll'
    }
}

# Create the scatter plot
fig = plot_main_figure(tdf)

#let's make your view!
app.layout = html.Div(id='site_body', children=[
        dcc.Graph(id='basic-interactions',figure=fig),
        html.Div(className='row', children=[
            html.Div([
                  dcc.Slider(
        id='dim-slider',
        min=1,
        max=200,
        step=1,
        value=e,
    ),
    html.Div(id='slider-output-container'),
                  dcc.Slider(
        id='tau-slider',
        min=1,
        max=100,
        step=1,
        value=tau_num,
    ),
    html.Div(id='tau-output-container'),
                     dcc.Slider(
        id='pt-slider',
        min=1,
        max=pred_interval,
        step=1,
        value=pred_interval,
    ),
    html.Div(id='pt-output-container'),
                     dcc.Slider(
        id='yoffset-slider',
        min=-1000,
        max=1000,
        step=100,
        value=yoffset),
    html.Div(id='yoffset-output-container'),
                dcc.Markdown("""
                    **Hover Data**

                    Mouse over values in the graph.
                """),
                html.Pre(id='hover-data', style=styles['pre'])
            ], className='three columns'),


            html.Div([
                dcc.Markdown("""
                    **Click Data**

                    Click on points in the graph.
                """),
                html.Pre(id='click-data', style=styles['pre']),
            ], className='three columns'),


            html.Div([
                dcc.Markdown("""
                    **Selection Data**

                    Choose the lasso or rectangle tool in the graph's menu
                    bar and then select points in the graph.

                    Note that if `layout.clickmode = 'event+select'`, selection data also 
                    accumulates (or un-accumulates) selected data if you hold down the shift
                    button while clicking.
                """),
                html.Pre(id='selected-data', style=styles['pre']),
            ], className='three columns'),

            html.Div([
                dcc.Markdown("""
                    **Zoom and Relayout Data**

                    Click and drag on the graph to zoom or click on the zoom
                    buttons in the graph's menu bar.
                    Clicking on legend items will also fire
                    this event.
                """),
                html.Pre(id='relayout-data', style=styles['pre']),
            ], className='three columns'),
            html.Div([
                html.Button('Shutdown Dash', id='bt_close', n_clicks=0), #tried using a button
                html.Div(id='close')
            ]),
        ])
])

@app.callback(
    Output('hover-data', 'children'),
    [Input('basic-interactions', 'hoverData')])
def display_hover_data(hoverData):
    '''
    Handles onHover event on data points in the scatter plot.
    '''
    return json.dumps(hoverData, indent=2)

@app.callback(
    dash.dependencies.Output('slider-output-container', 'children'),
    [dash.dependencies.Input('dim-slider', 'value')])
def update_output(value):
    return 'The number of embedding dimensions has been set to: "{}"'.format(value)

@app.callback(
    dash.dependencies.Output('tau-output-container', 'children'),
    [dash.dependencies.Input('tau-slider', 'value')])
def tautput(value):
    return 'The embedding offset (tau) has been set to: "{}"'.format(value)
    
@app.callback(
    dash.dependencies.Output('pt-output-container', 'children'),
    [dash.dependencies.Input('pt-slider', 'value')])
def pt_update(value):
    return 'The forecast interval has been set to: "{}"'.format(value)
        
@app.callback(
    dash.dependencies.Output('yoffset-output-container', 'children'),
    [dash.dependencies.Input('yoffset-slider', 'value')])
def yoffset_update(value):
    return 'Y values have been shifted by "{}"'.format(value)
        
@app.callback(
    dash.dependencies.Output('basic-interactions', 'figure'),
    [dash.dependencies.Input('dim-slider', 'value'), 
     dash.dependencies.Input('tau-slider', 'value'),
     dash.dependencies.Input('pt-slider', 'value'),
    dash.dependencies.Input('yoffset-slider', 'value')])
def update_graph(dim, tau, pt, new_yoffset):
    global e, tau_num, pred_interval, yoffset
    e = int(dim)
    tau_num = int(tau)
    yoffset = int(new_yoffset)
    pred_interval = pt
    import time
    time.sleep(2)
    return plot_main_figure(tdf)

@app.callback(
    Output('click-data', 'children'),
    [Input('basic-interactions', 'clickData')])
def display_click_data(clickData):
    '''
    Handles onClick event on a data point in the scatter plot.
    '''
    return json.dumps(clickData, indent=2)

@app.callback(
    Output('selected-data', 'children'),
    [Input('basic-interactions', 'selectedData')])
def display_selected_data(selectedData):
    '''
    Handles onSelect event on scatter plot.
    '''
    return json.dumps(selectedData, indent=2)

@app.callback(
    Output('relayout-data', 'children'),
    [Input('basic-interactions', 'relayoutData')])
def display_relayout_data(relayoutData):
    return json.dumps(relayoutData, indent=2)

@app.callback(
    Output("site_body", "children"), 
    Input('bt_close', 'n_clicks'))
def export_current_df(export_clicks): 
    '''
    Handles a button's onClick event
    '''
    if not dash.callback_context.triggered:
        raise dash.exceptions.PreventUpdate
    func = request.environ.get('werkzeug.server.shutdown')
    if func is None:
        raise RuntimeError('Not running with the Werkzeug Server')
    func()
    return 'Closed'

js = "<b style='color: red'>Please click on <a href='/proxy/" + str(proxy_port) + "/' target='_blank'>here</a> to open the dash</b>"
display(HTML(js))

app.run_server(mode="inline")
#app.run_server(debug=True, use_reloader=False, port=proxy_port)

In [ ]:
#poor man's message queue
old_e = e
old_tau_num = tau_num
old_pred_interval=pred_interval
old_yoffset = yoffset
def update_loop():
    global e, tau_num, pred_interval, old_yoffset, old_e, old_tau_num, old_pred_interval, old_yoffset
    try:
        if old_e != e or old_tau_num != tau_num or old_pred_interval != pred_interval or old_yoffset != yoffset:
            old_e = e
            old_tau_num = tau_num
            old_pred_interval=pred_interval
            old_yoffset = yoffset
            update_df(e=e,tau_num=tau_num,pred_interval=pred_interval)
            print(yoffset)
            
    except Exception as ex:
        print('Something went wrong. ¯\_(ツ)_/¯')
        print(ex)
        pass
    finally:
        import time
        time.sleep(1)
        update_loop()
update_loop()

0.5050999522209167 0.92253577709198
100
0.06655915081501007 0.5622630715370178
100
